In [1]:
import sqlite3
import pandas as pd
import numpy as np
import re
import datetime
import os
import requests
from bs4 import BeautifulSoup

In [2]:
#Drop columns function:
def drop (df, list_of_columns):   
    df1 = df.drop(list_of_columns, axis=1)
    return df1

In [3]:
#Replacement incorrect values - wrangling: col1,worth, col2,worthchange:
def replacement (df, col1, col2):
    df[col1].replace({"BUSD": ' '})
    df[col1].replace('na', '0.0', regex=True, inplace=True)
    df[col1].replace('[BUSD]+$', '', regex=True, inplace=True)
    df[col2].replace('[millions USD]+$', '', regex=True, inplace=True)
    df[col2].astype('float').astype('int')
    df[col2].astype('float').astype('int')
    return df

In [4]:
#Changing values - wrangling - gender:
def changing_text(df, column):
    df[column].replace('[M]+$', 'Male', regex=True, inplace=True)
    df[column].replace('[F]+$', 'Female', regex=True, inplace=True)
    df[column].astype('object').astype('str')
    return df

In [5]:
#Replacemente non values - wrangling - country:
def non_values(df, column):
    df[column].replace('None', np.nan, regex=True, inplace=True)
    df_no_null_values = df[df[column].isnull()] 
    df.drop(df[df[column].isnull()].index, inplace = True)
    return df

In [6]:
  #To expand the database I used a wikipedia page, data obtained through WebScrapping:

    #Extract URL - Scrapper:

#url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'  
def web_scraping(url):
    
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    table = soup.find_all('table',{'class':'wikitable sortable mw-datatable'})[0]

        # tr represent the table rows:
    rows = table.find_all('tr')
    rows_parsed = [row.text for row in rows]
    return rows_parsed

        #Parsering:

def smart_parser(row_text):
    row_text = row_text.replace('\nPopulation\n\n\n', '\nPopulation\n').strip('\n')
    row_text = row_text.replace('\n\n', '\n').strip('\n')
    row_text = row_text.replace('\nPopulation\n% of World\n', '\nPopulation % of World\n').strip('\n')
    row_text = re.sub('\[\d\]', '', row_text)
    
    return list(map(lambda x: x.strip(), row_text.split('\n')))

def well_parsed(rows_parsed):
    parsed = list(map(lambda x: smart_parser(x), rows_parsed))
    colnames = parsed[0]
    data = parsed[1:]
    return data

def wrg_scraping_data(data):  ##DUDA
    data['Country(or dependent territory)']=data['Country(or dependent territory)'].replace(['\[(.*?)\]','\(([^\)]+)\)'], ['',''], regex=True)
    data.columns = ['Rank', 'country','Population',' % of World Population','Date','Source']    
    data_scraping_clean = pd.DataFrame(data, columns=colnames)
    return data_scraping_clean
    
def exporting(dataset):
    data_scraping_clean=pd.merge(data_scraping_clean, df, on=['country', 'country'])      
    dataset_clean.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/df_scrapping.csv', index=False)
    return dataset_clean